In [1]:
import pandas as pd
import numpy as np
import regex as re

# Read the data

In [2]:
df = pd.read_csv('big_3.csv')
df.head()

,text,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"Indeed, I should have thought a little more. J...",NaN,NaN,NaN,NaN
1,"Then, how do you know?",NaN,NaN,NaN,NaN
2,"I see it, I deduce it. How do I know that you ...",NaN,NaN,NaN,NaN
3,"My dear Holmes, said I, ""this is too much. You...",NaN,NaN,NaN,NaN
4,"He chuckled to himself and rubbed his long, ne...",NaN,NaN,NaN,NaN


# Clean the data

In [3]:
clean_desc = []
df = df.astype(str) #changing all values' types to string

for w in range(len(df.text)):
    #making all words lowercase
    desc = df['text'][w].lower()
    #remove digits and special chars
    desc=re.sub("\d"," ",desc)
    #removing paranthesis
    desc = re.sub(r'\([^)]*\)', '', desc)
    # replacing one or more white space with a single white space
    desc = re.sub(r"\s+", " ", desc)
    
    clean_desc.append(desc)

#store clean version in new data object
df['text'] = clean_desc

data = df['text']
data.head()

0    indeed, i should have thought a little more. j...
1                               then, how do you know?
2    i see it, i deduce it. how do i know that you ...
3    my dear holmes, said i, "this is too much. you...
4    he chuckled to himself and rubbed his long, ne...
Name: text, dtype: object

# Store as list of lists of words

In [4]:
# store as list of lists of words
sentences_ted = []
for sent_str in data:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

In [5]:
import multiprocessing
from gensim.models import FastText

C:\Users\narin\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

# Initialize the model

In [7]:
model = FastText( vector_size=100, 
                  window=5, #how many words are around the world
                  min_count=1, #discard words less then frequency of 2
                  workers=cores-1,
                  sg=1)

# Build the vocabulary

In [8]:
model.build_vocab(sentences_ted, progress_per=10000) 

 # Train the model

In [9]:
model = FastText( sentences_ted,
                  vector_size=100, 
                  window=5, #how many words are around the world
                  min_count=1, #discard words less then frequency of 2
                  workers=cores-1,
                  sg=1)

# Connecting to Anvil Server

In [10]:
import anvil.server

anvil.server.connect("HVZVTIZOZ2YW54N7P7SRCQSM-VR54JGLMUIPSZGRA")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [11]:
@anvil.server.callable
def spellCheck(word):
    most_similar = model.wv.most_similar(word, topn=5)
    return most_similar